# Basic Model Fitting with rheo

This notebook demonstrates the basic workflow for fitting rheological models to experimental data using the rheo package.

We will:
1. Generate synthetic relaxation data
2. Fit three classical models (Maxwell, Zener, SpringPot)
3. Compare the fits visually
4. Display and interpret fitted parameters

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rheo.pipeline import Pipeline
from rheo.core.data import RheoData

# Set random seed for reproducibility
np.random.seed(42)

## Generate Synthetic Relaxation Data

We'll create synthetic stress relaxation data that resembles a viscoelastic polymer response.
The data follows a stretched exponential decay with added noise.

In [ ]:
# Generate time vector (logarithmic spacing)
time = np.logspace(-2, 3, 50)  # 0.01 to 1000 seconds

# Generate synthetic stress relaxation data
# Model: G(t) = G0 * exp(-(t/tau)^alpha) + Ge
G0 = 1e6  # Initial modulus (Pa)
tau = 10.0  # Relaxation time (s)
alpha = 0.7  # Stretched exponential parameter
Ge = 1e4  # Equilibrium modulus (Pa)

stress = G0 * np.exp(-(time / tau)**alpha) + Ge

# Add realistic noise (5% relative error)
noise_level = 0.05
stress_noisy = stress * (1 + noise_level * np.random.randn(len(stress)))

# Create RheoData object
data = RheoData(
    x=time,
    y=stress_noisy,
    x_units='s',
    y_units='Pa',
    domain='time',
    test_mode='relaxation'
)

print(f"Generated {len(time)} data points from {time[0]:.2e} to {time[-1]:.2e} seconds")
print(f"Stress range: {stress_noisy.min():.2e} to {stress_noisy.max():.2e} Pa")

## Visualize Raw Data

In [ ]:
plt.figure(figsize=(8, 5))
plt.loglog(time, stress_noisy, 'o', label='Experimental Data', alpha=0.6)
plt.loglog(time, stress, '--', label='True Model', color='gray', linewidth=2)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Stress Modulus G(t) (Pa)', fontsize=12)
plt.title('Synthetic Stress Relaxation Data', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Fit Three Classical Models

### 1. Maxwell Model

The Maxwell model represents a spring and dashpot in series:
$$G(t) = G_0 \exp(-t/\tau)$$

where $G_0$ is the initial modulus and $\tau = \eta/G_0$ is the relaxation time.

In [ ]:
# Create pipeline and fit Maxwell model
pipeline_maxwell = Pipeline()
pipeline_maxwell.load_data(data)
pipeline_maxwell.fit('maxwell')

# Get fitted parameters
maxwell_params = pipeline_maxwell.get_fitted_parameters()
print("\nMaxwell Model Parameters:")
for name, param in maxwell_params.items():
    print(f"  {name}: {param.value:.3e} {param.units}")

# Get predictions
maxwell_pred = pipeline_maxwell.predict(data)

### 2. Zener Model (Standard Linear Solid)

The Zener model (also called Standard Linear Solid) has two moduli and one relaxation time:
$$G(t) = G_e + G_m \exp(-t/\tau)$$

This model includes an equilibrium modulus $G_e$, making it more suitable for materials that don't fully relax.

In [ ]:
# Fit Zener model
pipeline_zener = Pipeline()
pipeline_zener.load_data(data)
pipeline_zener.fit('zener')

# Get fitted parameters
zener_params = pipeline_zener.get_fitted_parameters()
print("\nZener Model Parameters:")
for name, param in zener_params.items():
    print(f"  {name}: {param.value:.3e} {param.units}")

# Get predictions
zener_pred = pipeline_zener.predict(data)

### 3. SpringPot Model

The SpringPot model is a fractional viscoelastic element:
$$G(t) = C_{\alpha} \frac{t^{-\alpha}}{\Gamma(1-\alpha)}$$

where $\alpha$ is the fractional order (0 = spring, 1 = dashpot), and $C_{\alpha}$ is the quasi-property.

In [ ]:
# Fit SpringPot model
pipeline_springpot = Pipeline()
pipeline_springpot.load_data(data)
pipeline_springpot.fit('springpot')

# Get fitted parameters
springpot_params = pipeline_springpot.get_fitted_parameters()
print("\nSpringPot Model Parameters:")
for name, param in springpot_params.items():
    print(f"  {name}: {param.value:.3e} {param.units if param.units else 'dimensionless'}")

# Get predictions
springpot_pred = pipeline_springpot.predict(data)

## Compare Fits Visually

In [ ]:
plt.figure(figsize=(10, 6))

# Plot data and fits
plt.loglog(time, stress_noisy, 'o', label='Experimental Data', 
           color='black', markersize=6, alpha=0.5)
plt.loglog(time, maxwell_pred, '-', label='Maxwell', linewidth=2)
plt.loglog(time, zener_pred, '-', label='Zener', linewidth=2)
plt.loglog(time, springpot_pred, '-', label='SpringPot', linewidth=2)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Stress Modulus G(t) (Pa)', fontsize=12)
plt.title('Model Comparison: Stress Relaxation Fits', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Residual Analysis

In [ ]:
# Calculate residuals
maxwell_residuals = stress_noisy - maxwell_pred
zener_residuals = stress_noisy - zener_pred
springpot_residuals = stress_noisy - springpot_pred

# Plot residuals
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, residuals, name in zip(axes, 
                                [maxwell_residuals, zener_residuals, springpot_residuals],
                                ['Maxwell', 'Zener', 'SpringPot']):
    ax.semilogx(time, residuals / stress_noisy * 100, 'o-', alpha=0.6)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
    ax.set_xlabel('Time (s)', fontsize=10)
    ax.set_ylabel('Residual (%)', fontsize=10)
    ax.set_title(f'{name} Residuals', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Quantitative Comparison

In [ ]:
# Calculate R² and RMSE for each model
def calculate_metrics(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r_squared = 1 - (ss_res / ss_tot)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    return r_squared, rmse

maxwell_r2, maxwell_rmse = calculate_metrics(stress_noisy, maxwell_pred)
zener_r2, zener_rmse = calculate_metrics(stress_noisy, zener_pred)
springpot_r2, springpot_rmse = calculate_metrics(stress_noisy, springpot_pred)

print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
print(f"\nMaxwell Model:")
print(f"  R² = {maxwell_r2:.4f}")
print(f"  RMSE = {maxwell_rmse:.2e} Pa")

print(f"\nZener Model:")
print(f"  R² = {zener_r2:.4f}")
print(f"  RMSE = {zener_rmse:.2e} Pa")

print(f"\nSpringPot Model:")
print(f"  R² = {springpot_r2:.4f}")
print(f"  RMSE = {springpot_rmse:.2e} Pa")

# Determine best model
models = ['Maxwell', 'Zener', 'SpringPot']
r2_values = [maxwell_r2, zener_r2, springpot_r2]
best_model_idx = np.argmax(r2_values)

print(f"\n" + "="*60)
print(f"Best Model: {models[best_model_idx]} (R² = {r2_values[best_model_idx]:.4f})")
print("="*60)

## Conclusion

In this notebook, we demonstrated the basic workflow for fitting rheological models using the rheo package:

1. **Data Generation**: Created synthetic stress relaxation data with realistic noise
2. **Model Fitting**: Fitted three classical models (Maxwell, Zener, SpringPot) using the Pipeline API
3. **Visual Comparison**: Compared model predictions against experimental data
4. **Residual Analysis**: Examined residuals to assess fit quality
5. **Quantitative Metrics**: Calculated R² and RMSE for objective comparison

### Key Takeaways:

- The **Zener model** typically performs best for materials with non-zero equilibrium modulus
- The **Maxwell model** is suitable for materials that fully relax but may struggle with complex relaxation behaviors
- The **SpringPot model** captures power-law behavior but may not capture equilibrium effects

### Next Steps:

- Explore fractional models for more complex relaxation behaviors (see `advanced_workflows.ipynb`)
- Use `ModelComparisonPipeline` for systematic model comparison (see `multi_model_comparison.ipynb`)
- Apply time-temperature superposition for multi-temperature data (see `mastercurve_generation.ipynb`)